# Neural Networks
> Neural networks can be constructed using the torch.nn package.

A typical training procedure for a neural network is as follows:
-  Iterate over a dataset of inputs
-  Process input through the network
-  Compute the loss (how far is the output from being correct)
-  Propagate gradients back into the network’s parameters
-  Update the weights of the network, typically using a simple update rule: 
    ```weight = weight - learning_rate * gradient```


### Define the network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1,6,5) 
        self.conv2 = nn.Conv2d(6,16,5) 
        
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self,X):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(X)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [19]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


> You just have to define the ```forward``` function, and the ```backward``` function (where gradients are computed) is automatically defined for you using ```autograd```. You can use any of the Tensor operations in the ```forward``` function. <br><br>
> The learnable parameters of a model are returned by ```net.parameters()```

In [31]:
params = list(net.parameters())
print(len(params))
print(params[0].size())
# print(params)

10
torch.Size([6, 1, 5, 5])


In [23]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor(1.00000e-02 *
       [[ 0.9295, -3.7588, -6.1186, -0.8479,  6.6543, -6.0397, -9.2455,
         -2.0642,  0.8355,  0.0652]])


> Zero the gradient buffers of all parameters and backprops with random gradients:

In [24]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<br><br>
- ```torch.Tensor``` - A multi-dimensional array with support for autograd operations like ```backward()```. Also holds the gradient w.r.t. the tensor. 
<br><br>
- ```nn.Module``` - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
<br><br>
- ```nn.Parameter``` - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a ```Module```. 
<br><br>
- ```autograd.Function``` - Implements forward and backward definitions of an autograd operation. Every ```Tensor``` operation, creates at least a single ```Function``` node, that connects to functions that created a ```Tensor``` and encodes its history.
<br><br>

### Loss Function

In [25]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6176)


> input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d <br>
      -> view -> linear -> relu -> linear -> relu -> linear <br>
      -> MSELoss <br>
      -> loss

In [26]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### Backprop

In [27]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor(1.00000e-03 *
       [-5.5786,  2.1472, -3.0126, -5.8203, -4.0826, -3.1116])


### Update the weights

In [28]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [29]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update